In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
import plotly_express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from downcast import reduce
import warnings
from plotly.io import to_image
from IPython.display import Image

warnings.filterwarnings('ignore')
%matplotlib inline

In [3]:
prices = pd.read_pickle('../data/processed/item_prices.pkl')
sales = pd.read_pickle('../data/processed/item_sales.pkl')
calendar = pd.read_pickle('../data/processed/daily_calendar_with_events.pkl')


In [16]:
calendar['date'] = pd.to_datetime(calendar['date'])
calendar['year'] = calendar['date'].dt.year
calendar['month'] = calendar['date'].dt.month
calendar['day'] = calendar['date'].dt.day
calendar.head()

,date,weekday,weekday_int,d,event,year,month,day
0,2011-01-29,Saturday,1,d_1,NaN,2011,1,29
1,2011-01-30,Sunday,2,d_2,NaN,2011,1,30
2,2011-01-31,Monday,3,d_3,NaN,2011,1,31
3,2011-02-01,Tuesday,4,d_4,NaN,2011,2,1
4,2011-02-02,Wednesday,5,d_5,NaN,2011,2,2


### Sazonalidade Semanal

> Calcular as vendas totais de cada região e somar as vendas em cada dia da semana.
 

In [7]:
sales_df = sales.groupby(['region']).sum()
sales_df.head(100)

,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
region,,,,,,,,,,,,,,,,,,,,,
Boston,9438,9630,6778,7381,5912,9006,6226,9440,9376,7319,...,10800,12460,13709,9868,10922,9600,9602,10615,12266,12282
New York,14195,13805,10108,11047,9925,11322,12251,16610,14696,11822,...,16255,20564,23032,17052,15784,15148,14488,17095,21834,23187
Philadelphia,8998,8314,6897,6984,3309,8883,9533,11882,8664,6431,...,14734,15338,14899,11139,10864,10595,10943,12807,14862,14326


In [13]:
days = list(calendar['weekday'].iloc[:1913])

weekly_agg = pd.DataFrame({'weeks': days, 'BOS' : sales_df.iloc[0], 'NY' : sales_df.iloc[1], 'PHI' : sales_df.iloc[2]})
weekly_agg = weekly_agg.groupby(['weeks']).mean()
#weekly_agg.head()

fig = go.Figure()
fig.add_trace(go.Scatter(x=days, y=weekly_agg['BOS'],
                    mode='lines',
                    name='Boston'))
                    
fig.add_trace(go.Scatter(x=days, y= weekly_agg['NY'],
                    mode='lines',
                    name='New York'))

fig.add_trace(go.Scatter(x=days, y=weekly_agg['PHI'],
                    mode='lines', name='Philadelphia'))

fig.update_layout(title='Weekly Trend',
                   xaxis_title='Time',
                   yaxis_title='Sales')
fig.show()

### Observações
> Tendência semelhante nas três regiões os picos de venda acontecem no começo da semana (Segunda-feira e Terça-feira).

### Sazonalidade Mensal

> Calcular as vendas totais de cada região e somar as vendas em cada mês do ano.
 

In [24]:
month_no = list(calendar['month'].iloc[:1913])

monthly_agg = pd.DataFrame({'months': month_no, 'BOS' : sales_df.iloc[0], 'NY' : sales_df.iloc[1], 'PHI' : sales_df.iloc[2]})
monthly_agg = monthly_agg.groupby(['months']).mean()

months = ['Jan','Feb','March','April','May','June','July','Aug','Sept','Oct','Nov','Dec']

fig = go.Figure()
fig.add_trace(go.Scatter(x=months, y=monthly_agg['BOS'],
                    mode='lines',
                    name='BOS'))
fig.add_trace(go.Scatter(x=months, y=monthly_agg['NY'],
                    mode='lines',
                    name='NY'))
fig.add_trace(go.Scatter(x=months, y=monthly_agg['PHI'],
                    mode='lines', name='PHI'))

fig.update_layout(title='Monthly Trend',
                   xaxis_title='Time',
                   yaxis_title='Sales')

fig.show()

### Observações

> Maiores vendas observadas em Agosto (Aug), com pequenas quedas no mês de Novembro (Nov) e Maio (May).


### Sazonalidade Semanal por categoria e região

#### Região de New York

In [63]:
sales['department'].unique()

['ACCESORIES_1', 'ACCESORIES_2', 'HOME_&_GARDEN_1', 'HOME_&_GARDEN_2', 'SUPERMARKET_1', 'SUPERMARKET_2', 'SUPERMARKET_3']
Categories (7, object): ['ACCESORIES_1', 'ACCESORIES_2', 'HOME_&_GARDEN_1', 'HOME_&_GARDEN_2', 'SUPERMARKET_1', 'SUPERMARKET_2', 'SUPERMARKET_3']

In [66]:
sales_df = sales[sales['region'] == 'New York'].groupby(['department']).sum()

,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
department,,,,,,,,,,,,,,,,,,,,,
ACCESORIES_1,1739,1502,1422,1352,1139,1427,1281,1920,1419,1471,...,1496,2041,2146,1607,1717,1508,1457,1838,2203,2135
ACCESORIES_2,63,59,50,53,42,32,33,66,63,37,...,162,135,184,99,132,138,133,177,164,222
HOME_&_GARDEN_1,1527,1515,1116,1148,935,1014,1224,1834,1496,1089,...,2984,4064,4490,3158,2949,2709,2597,3116,4220,4626
HOME_&_GARDEN_2,765,867,576,630,631,593,708,920,741,588,...,881,1230,1357,971,898,930,822,973,1402,1398
SUPERMARKET_1,1157,1142,787,743,729,796,913,1036,1049,804,...,1532,1562,1433,1126,1119,1216,1097,1474,1802,1529
SUPERMARKET_2,1864,1987,1315,1452,1239,1427,1554,1962,1928,1787,...,1656,2091,2537,2028,1753,1753,1554,1821,2284,2836
SUPERMARKET_3,7080,6733,4842,5669,5210,6033,6538,8872,8000,6046,...,7544,9441,10885,8063,7216,6894,6828,7696,9759,10441


In [68]:
days = list(calendar['weekday'].iloc[:1913])

depts = sales['department'].unique()

w_seas = pd.DataFrame({ 'days': days, 
                        'ACCESORIES_1' : sales_df.iloc[0], 
                        'ACCESORIES_2' : sales_df.iloc[1], 
                        'HOME_&_GARDEN_1' : sales_df.iloc[2], 
                        'HOME_&_GARDEN_2' : sales_df.iloc[3], 
                        'SUPERMARKET_1' : sales_df.iloc[4], 
                        'SUPERMARKET_2' : sales_df.iloc[5], 
                        'SUPERMARKET_3' : sales_df.iloc[6] })

w_seas = w_seas.groupby(['days']).mean()
w_seas['days'] = ['Sat','Sun','Mon','Tue','Wed','Thurs','Fri']

fig = go.Figure()
for i in depts:
    fig.add_trace(go.Scatter(x= w_seas['days'], y= w_seas[i],
                    mode='lines',
                    name=i))

fig.update_layout(title='Weekly Seasonalities of depts in New York',
                   xaxis_title='Time',
                   yaxis_title='Sales')
fig.show()

#### Região de Boston

In [71]:
sales_df = sales[sales['region'] == 'Boston'].groupby(['department']).sum()

,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
department,,,,,,,,,,,,,,,,,,,,,
ACCESORIES_1,837,775,435,756,448,798,355,732,725,496,...,942,972,1273,725,867,868,756,921,1073,1139
ACCESORIES_2,42,95,91,53,53,33,35,53,69,28,...,165,152,188,105,158,112,126,135,149,127
HOME_&_GARDEN_1,1281,1261,845,820,598,801,843,1142,1200,937,...,2135,2696,2822,1809,2278,1881,2030,2245,2523,2588
HOME_&_GARDEN_2,425,469,283,282,211,307,322,458,462,353,...,564,696,709,395,553,497,523,475,672,663
SUPERMARKET_1,500,446,381,317,297,445,337,466,432,368,...,730,796,736,589,613,644,707,695,748,650
SUPERMARKET_2,1384,1426,1008,1316,1143,1826,917,1319,1280,1111,...,1189,1257,1451,1292,1268,1168,1128,1115,1085,1341
SUPERMARKET_3,4969,5158,3735,3837,3162,4796,3417,5270,5208,4026,...,5075,5891,6530,4953,5185,4430,4332,5029,6016,5774


In [72]:
days = list(calendar['weekday'].iloc[:1913])

depts = sales['department'].unique()

w_seas = pd.DataFrame({ 'days': days, 
                        'ACCESORIES_1' : sales_df.iloc[0], 
                        'ACCESORIES_2' : sales_df.iloc[1], 
                        'HOME_&_GARDEN_1' : sales_df.iloc[2], 
                        'HOME_&_GARDEN_2' : sales_df.iloc[3], 
                        'SUPERMARKET_1' : sales_df.iloc[4], 
                        'SUPERMARKET_2' : sales_df.iloc[5], 
                        'SUPERMARKET_3' : sales_df.iloc[6] })

w_seas = w_seas.groupby(['days']).mean()
w_seas['days'] = ['Sat','Sun','Mon','Tue','Wed','Thurs','Fri']

fig = go.Figure()
for i in depts:
    fig.add_trace(go.Scatter(x=w_seas['days'], y= w_seas[i],
                    mode='lines',
                    name=i))

fig.update_layout(title='Weekly Seasonalities of depts in Boston',
                   xaxis_title='Time',
                   yaxis_title='Sales')
fig.show()

### Região de Philadelphia

In [73]:
sales_df = sales[sales['region'] == 'Philadelphia'].groupby(['department']).sum()

,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
department,,,,,,,,,,,,,,,,,,,,,
ACCESORIES_1,1034,895,640,423,127,908,1219,1179,597,547,...,929,1207,906,706,836,765,801,919,1121,952
ACCESORIES_2,49,31,44,32,5,22,21,36,26,36,...,92,127,123,81,77,81,80,95,77,108
HOME_&_GARDEN_1,1297,1082,866,764,269,849,1094,1358,798,831,...,2256,2493,2408,1809,1759,1803,1953,2401,2802,2581
HOME_&_GARDEN_2,394,440,241,221,85,334,385,483,278,258,...,501,542,537,443,398,419,438,518,629,602
SUPERMARKET_1,686,628,489,448,183,656,653,733,444,414,...,930,1165,893,746,812,797,835,1000,1162,1006
SUPERMARKET_2,846,796,851,838,487,1122,878,1103,1065,786,...,2816,2444,2600,1927,1921,1813,1764,2038,2155,2296
SUPERMARKET_3,4692,4442,3766,4258,2153,4992,5283,6990,5456,3559,...,7210,7360,7432,5427,5061,4917,5072,5836,6916,6781


In [74]:
days = list(calendar['weekday'].iloc[:1913])

depts = sales['department'].unique()

w_seas = pd.DataFrame({ 'days': days, 
                        'ACCESORIES_1' : sales_df.iloc[0], 
                        'ACCESORIES_2' : sales_df.iloc[1], 
                        'HOME_&_GARDEN_1' : sales_df.iloc[2], 
                        'HOME_&_GARDEN_2' : sales_df.iloc[3], 
                        'SUPERMARKET_1' : sales_df.iloc[4], 
                        'SUPERMARKET_2' : sales_df.iloc[5], 
                        'SUPERMARKET_3' : sales_df.iloc[6] })

w_seas = w_seas.groupby(['days']).mean()
w_seas['days'] = ['Sat','Sun','Mon','Tue','Wed','Thurs','Fri']

fig = go.Figure()
for i in depts:
    fig.add_trace(go.Scatter(x=w_seas['days'], y= w_seas[i],
                    mode='lines',
                    name=i))

fig.update_layout(title='Weekly Seasonalities of depts in Philadelphia',
                   xaxis_title='Time',
                   yaxis_title='Sales')
fig.show()

### Observações

> - As tendências semanais para todas as regiões e departamentos foram quase as mesmas
> - As vendas nos finais de semana foram maiores para todos os departamentos 
> - É possivel observar uma queda nas vendas nos mês de **Maio** (May) em todos os departamentos independente de região 
> - Em **Boston** e **New York** as maiores vendas foram observadas em **Agosto** (Aug) e para **Philadelphia** foi observada a maior quantidade de vendas em **Fevereiro** (Feb)
